In [1]:
import os, json
from plugins.utiles import *
from binance.client import Client

api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_API_SECRET")

client = Client(api_key, api_secret)

# USDC PRICE
px = float(client.get_symbol_ticker(symbol="USDCUSDT")["price"])
px


with open("dags/data/memory.json", "r") as f:
    memory = json.load(f)

In [2]:
data = get_market_data(client, limit=12, symbol="ETHUSDC")
data

{'symbol': 'ETHUSDC',
 'tf': '1h',
 'fees_bps': 7,
 'features': {'ohlcv_stats': {'change_1h_pct': 0.36,
   'change_3h_pct': 0.19,
   'change_12h_pct': 0.3,
   'high_12h': 4492.2,
   'low_12h': 4359.24,
   'avg_range': 42.26,
   'vol_sum': 75333.4491,
   'vol_avg': 6277.7874,
   'green_bars': 7,
   'red_bars': 5,
   'longest_green_streak': 5,
   'longest_red_streak': 4},
  'hourly_snapshots': [{'time': 1756695600,
    'close': 4405.18,
    'ema20': 4434.52,
    'ema50': 4431.47,
    'rsi14': 44.17,
    'macd': -6.098,
    'macd_sig': 5.088,
    'macd_hist': -11.186,
    'atr_pct': 0.89,
    'above_ema': False,
    'rsi_trend': 'flat'},
   {'time': 1756699200,
    'close': 4386.53,
    'ema20': 4429.95,
    'ema50': 4429.71,
    'rsi14': 41.08,
    'macd': -8.964,
    'macd_sig': 2.277,
    'macd_hist': -11.241,
    'atr_pct': 0.9,
    'above_ema': False,
    'rsi_trend': 'flat'},
   {'time': 1756702800,
    'close': 4395.51,
    'ema20': 4426.67,
    'ema50': 4428.37,
    'rsi14': 43.14

In [3]:
memory.setdefault("capital", {})
memory.setdefault("recent_decisions", [])
memory.setdefault("capital_history", [])

# --- 2) Capital & balances (initial depuis mémoire si présent) ---
initial_cap = float(memory["capital"].get("initial", 5000.0))
capital, balances = get_capital_and_balances(client, INITIAL_CAPITAL_USDC=initial_cap)

# --- 3) MAJ état capital : snapshot -> DD 7j -> halt ---
snapshot_capital(memory, capital["current"])  # ajoute le point du run (anti-doublon horaire)
capital["max_dd_7d"] = compute_max_dd_7d(memory.get("capital_history", []), capital["current"])
capital["halt_triggered"] = bool(capital["current"] < 0.5 * capital["initial"])

# --- 4) Performance (à partir des décisions conservées) ---
recent_decisions = memory.get("recent_decisions", [])
performance = compute_performance(recent_decisions, capital["current"])

# --- 5) État mémoire consolidé (sert au sizing paliers) ---
memory["capital"] = capital
memory["balances"] = balances
memory["recent_decisions"] = recent_decisions
memory["performance"] = performance

# --- 6) Market data avec bloc sizing "paliers" (BUY borné par headroom & rebalance, SELL non capé) ---
md_eth = get_market_data(client, "ETHUSDC", balances=balances, memory=memory)
md_btc = get_market_data(client, "BTCUSDC", balances=balances, memory=memory)
market_data = {"ETHUSDC": md_eth, "BTCUSDC": md_btc}

# --- 7) Payload, persistance snapshot & XCom ---
payload = {"market_data": market_data, "memory": memory}

payload

{'market_data': {'ETHUSDC': {'symbol': 'ETHUSDC',
   'tf': '1h',
   'fees_bps': 7,
   'features': {'ohlcv_stats': {'change_1h_pct': 0.37,
     'change_3h_pct': 0.21,
     'change_12h_pct': 0.32,
     'high_12h': 4492.2,
     'low_12h': 4359.24,
     'avg_range': 42.26,
     'vol_sum': 75334.2272,
     'vol_avg': 6277.8523,
     'green_bars': 7,
     'red_bars': 5,
     'longest_green_streak': 5,
     'longest_red_streak': 4},
    'hourly_snapshots': [{'time': 1756695600,
      'close': 4405.18,
      'ema20': 4434.52,
      'ema50': 4431.47,
      'rsi14': 44.17,
      'macd': -6.098,
      'macd_sig': 5.088,
      'macd_hist': -11.186,
      'atr_pct': 0.89,
      'above_ema': False,
      'rsi_trend': 'flat'},
     {'time': 1756699200,
      'close': 4386.53,
      'ema20': 4429.95,
      'ema50': 4429.71,
      'rsi14': 41.08,
      'macd': -8.964,
      'macd_sig': 2.277,
      'macd_hist': -11.241,
      'atr_pct': 0.9,
      'above_ema': False,
      'rsi_trend': 'flat'},
     {'

In [4]:
md = payload["market_data"]
for sym in ("ETHUSDC", "BTCUSDC"):
    s = md[sym]["features"]["sizing"]
    print(sym, "base_free=", s["base_free"], "px=", s["px_est"])
    for p in s["paliers_sell"]:
        print(p["pct"], "% → qty=", p["qty_base"], "notional=", p["notional_usdc"], "ok=", p["feasible"], p["why_not"])

ETHUSDC base_free= 0.00162498 px= 4402.76
5 % → qty= 0.0 notional= 0.0 ok= False rounded_to_zero,below_min_notional
10 % → qty= 0.0001 notional= 0.44 ok= False below_min_notional
25 % → qty= 0.0004 notional= 1.76 ok= False below_min_notional
50 % → qty= 0.0008 notional= 3.52 ok= False below_min_notional
75 % → qty= 0.0012 notional= 5.28 ok= True 
100 % → qty= 0.0016 notional= 7.04 ok= True 
BTCUSDC base_free= 4.151e-05 px= 109216.0
5 % → qty= 0.0 notional= 0.0 ok= False rounded_to_zero,below_min_notional
10 % → qty= 0.0 notional= 0.0 ok= False rounded_to_zero,below_min_notional
25 % → qty= 1e-05 notional= 1.09 ok= False below_min_notional
50 % → qty= 2e-05 notional= 2.18 ok= False below_min_notional
75 % → qty= 3e-05 notional= 3.28 ok= False below_min_notional
100 % → qty= 4e-05 notional= 4.37 ok= False below_min_notional


In [2]:
payload = {
  "market_data": {
    "ETHUSDC": {
      "symbol": "ETHUSDC",
      "tf": "1h",
      "fees_bps": 7,
      "features": {
        "ohlcv_stats": {
          "change_1h_pct": 0.8,
          "change_3h_pct": 2.4,
          "change_12h_pct": 4.1,
          "high_12h": 4025.0,
          "low_12h": 3830.0,
          "avg_range": 28.5,
          "vol_sum": 92000.0,
          "vol_avg": 7666.7,
          "green_bars": 9,
          "red_bars": 3,
          "longest_green_streak": 5,
          "longest_red_streak": 2
        },
        "hourly_snapshots": [
          {"time": 1756728000, "close": 3988.0, "ema20": 3965.0, "ema50": 3948.0, "rsi14": 55.2, "macd": 8.2, "macd_sig": 4.1, "macd_hist": 4.1, "atr_pct": 0.82, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756731600, "close": 4002.0, "ema20": 3976.0, "ema50": 3957.0, "rsi14": 57.0, "macd": 10.4, "macd_sig": 5.6, "macd_hist": 4.8, "atr_pct": 0.83, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756735200, "close": 4010.0, "ema20": 3986.0, "ema50": 3965.0, "rsi14": 58.6, "macd": 12.0, "macd_sig": 6.8, "macd_hist": 5.2, "atr_pct": 0.85, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756738800, "close": 4018.0, "ema20": 3996.0, "ema50": 3973.0, "rsi14": 59.8, "macd": 13.5, "macd_sig": 7.9, "macd_hist": 5.6, "atr_pct": 0.86, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756742400, "close": 4020.0, "ema20": 4005.0, "ema50": 3981.0, "rsi14": 60.4, "macd": 14.8, "macd_sig": 8.7, "macd_hist": 6.1, "atr_pct": 0.86, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756746000, "close": 4015.0, "ema20": 4010.0, "ema50": 3986.0, "rsi14": 59.2, "macd": 14.0, "macd_sig": 9.1, "macd_hist": 4.9, "atr_pct": 0.85, "above_ema": True, "rsi_trend": "flat"},
          {"time": 1756749600, "close": 4018.0, "ema20": 4013.0, "ema50": 3990.0, "rsi14": 59.8, "macd": 14.3, "macd_sig": 9.5, "macd_hist": 4.8, "atr_pct": 0.84, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756753200, "close": 4022.0, "ema20": 4016.0, "ema50": 3994.0, "rsi14": 60.5, "macd": 15.0, "macd_sig": 10.0, "macd_hist": 5.0, "atr_pct": 0.85, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756756800, "close": 4026.0, "ema20": 4020.0, "ema50": 3998.0, "rsi14": 61.1, "macd": 15.6, "macd_sig": 10.5, "macd_hist": 5.1, "atr_pct": 0.86, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756760400, "close": 4030.0, "ema20": 4024.0, "ema50": 4003.0, "rsi14": 61.8, "macd": 16.2, "macd_sig": 11.0, "macd_hist": 5.2, "atr_pct": 0.86, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756764000, "close": 4032.0, "ema20": 4027.0, "ema50": 4007.0, "rsi14": 62.2, "macd": 16.8, "macd_sig": 11.5, "macd_hist": 5.3, "atr_pct": 0.87, "above_ema": True, "rsi_trend": "up"},
          {"time": 1756767600, "close": 4034.0, "ema20": 4030.0, "ema50": 4011.0, "rsi14": 62.7, "macd": 17.4, "macd_sig": 12.0, "macd_hist": 5.4, "atr_pct": 0.87, "above_ema": True, "rsi_trend": "up"}
        ],
        "stats": {
          "ema20": {"last": 4030.0, "min": 3965.0, "max": 4030.0, "mean": 4007.3, "slope": 1.25},
          "ema50": {"last": 4011.0, "min": 3948.0, "max": 4011.0, "mean": 3981.9, "slope": 1.05},
          "rsi14": {"last": 62.7, "min": 55.2, "max": 62.7, "mean": 59.0, "slope": 0.75, "above50_cnt": 12},
          "macd": {"last": 17.4, "signal_last": 12.0, "hist_last": 5.4, "hist_mean": 4.9},
          "atr_pct": {"last": 0.87, "mean": 0.86},
          "h1": {"dist_ema50_pct": 0.57, "hh": 4034.0, "ll": 3988.0, "consec_hist_pos": 8, "hist_slope_3": 0.6}
        },
        "position_state": {
          "status": "flat",
          "size_base": 0.0,
          "size_quote": 0.0,
          "px": 4034.0,
          "lot_size": 0.0001,
          "min_notional": 5.0,
          "exposure_pct": 0.0
        },
        "sizing": {
          "mode": "paliers",
          "fees_bps": 7,
          "usdc_free": 150.0,
          "base_free": 0.0,
          "cap_current_usdc": 200.0,
          "px_est": 4034.0,
          "constraints": {
            "lot_size": 0.0001,
            "min_notional_usdc": 5.0,
            "max_rebalance_pct": 0.30,
            "max_exposure_pct": 0.25,
            "headroom_usdc": 50.0
          },
          "paliers_buy": [
            {"pct": 5,  "budget_plan_usdc": 7.5,  "budget_used_usdc": 7.5,  "qty_base": 0.0018, "notional_usdc": 7.26, "feasible": True,  "why_not": ""},
            {"pct": 10, "budget_plan_usdc": 15.0, "budget_used_usdc": 15.0, "qty_base": 0.0037, "notional_usdc": 14.92, "feasible": True,  "why_not": ""},
            {"pct": 25, "budget_plan_usdc": 37.5, "budget_used_usdc": 37.5, "qty_base": 0.0092, "notional_usdc": 37.11, "feasible": True,  "why_not": ""},
            {"pct": 50, "budget_plan_usdc": 75.0, "budget_used_usdc": 50.0, "qty_base": 0.0124, "notional_usdc": 49.62, "feasible": True,  "why_not": ""},
            {"pct": 75, "budget_plan_usdc": 112.5,"budget_used_usdc": 50.0, "qty_base": 0.0124, "notional_usdc": 49.62, "feasible": True,  "why_not": ""}
          ],
          "paliers_sell": [
            {"pct": 5,  "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 10, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 25, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 50, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 75, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"}
          ],
          "trade_ticket": {"max_buy_usdc": 50.0, "max_buy_qty_base": 0.0124, "max_sell_qty_base": 0.0}
        }
      }
    },
    "BTCUSDC": {
      "symbol": "BTCUSDC",
      "tf": "1h",
      "fees_bps": 7,
      "features": {
        "ohlcv_stats": {
          "change_1h_pct": 0.1,
          "change_3h_pct": 0.3,
          "change_12h_pct": -0.4,
          "high_12h": 110500.0,
          "low_12h": 108100.0,
          "avg_range": 520.0,
          "vol_sum": 1900.0,
          "vol_avg": 158.3,
          "green_bars": 6,
          "red_bars": 6,
          "longest_green_streak": 3,
          "longest_red_streak": 3
        },
        "hourly_snapshots": [
          {"time": 1756767600, "close": 109200.0, "ema20": 109250.0, "ema50": 109420.0, "rsi14": 49.5, "macd": -5.0, "macd_sig": -6.0, "macd_hist": 1.0, "atr_pct": 0.48, "above_ema": False, "rsi_trend": "flat"},
          {"time": 1756764000, "close": 109150.0, "ema20": 109240.0, "ema50": 109430.0, "rsi14": 49.0, "macd": -7.0, "macd_sig": -7.5, "macd_hist": 0.5, "atr_pct": 0.49, "above_ema": False, "rsi_trend": "down"},
          {"time": 1756760400, "close": 109180.0, "ema20": 109260.0, "ema50": 109460.0, "rsi14": 49.2, "macd": -8.0, "macd_sig": -7.8, "macd_hist": -0.2, "atr_pct": 0.50, "above_ema": False, "rsi_trend": "flat"},
          {"time": 1756756800, "close": 109250.0, "ema20": 109300.0, "ema50": 109480.0, "rsi14": 50.1, "macd": -6.0, "macd_sig": -7.0, "macd_hist": 1.0, "atr_pct": 0.52, "above_ema": False, "rsi_trend": "up"},
          {"time": 1756753200, "close": 109300.0, "ema20": 109320.0, "ema50": 109500.0, "rsi14": 50.0, "macd": -5.0, "macd_sig": -6.5, "macd_hist": 1.5, "atr_pct": 0.51, "above_ema": False, "rsi_trend": "up"},
          {"time": 1756749600, "close": 109100.0, "ema20": 109250.0, "ema50": 109520.0, "rsi14": 48.7, "macd": -9.0, "macd_sig": -8.0, "macd_hist": -1.0, "atr_pct": 0.52, "above_ema": False, "rsi_trend": "down"},
          {"time": 1756746000, "close": 109050.0, "ema20": 109230.0, "ema50": 109530.0, "rsi14": 48.1, "macd": -10.0, "macd_sig": -8.5, "macd_hist": -1.5, "atr_pct": 0.51, "above_ema": False, "rsi_trend": "down"},
          {"time": 1756742400, "close": 109000.0, "ema20": 109210.0, "ema50": 109520.0, "rsi14": 47.8, "macd": -11.0, "macd_sig": -9.0, "macd_hist": -2.0, "atr_pct": 0.50, "above_ema": False, "rsi_trend": "down"},
          {"time": 1756738800, "close": 109100.0, "ema20": 109230.0, "ema50": 109510.0, "rsi14": 48.5, "macd": -10.5, "macd_sig": -9.2, "macd_hist": -1.3, "atr_pct": 0.50, "above_ema": False, "rsi_trend": "flat"},
          {"time": 1756735200, "close": 109050.0, "ema20": 109240.0, "ema50": 109520.0, "rsi14": 48.0, "macd": -11.0, "macd_sig": -9.5, "macd_hist": -1.5, "atr_pct": 0.50, "above_ema": False, "rsi_trend": "down"},
          {"time": 1756731600, "close": 108980.0, "ema20": 109230.0, "ema50": 109540.0, "rsi14": 47.5, "macd": -12.5, "macd_sig": -10.3, "macd_hist": -2.2, "atr_pct": 0.49, "above_ema": False, "rsi_trend": "down"},
          {"time": 1756728000, "close": 108950.0, "ema20": 109220.0, "ema50": 109550.0, "rsi14": 47.3, "macd": -13.0, "macd_sig": -10.8, "macd_hist": -2.2, "atr_pct": 0.49, "above_ema": False, "rsi_trend": "down"}
        ],
        "stats": {
          "ema20": {"last": 109250.0, "min": 109210.0, "max": 109320.0, "mean": 109250.0, "slope": 0.0},
          "ema50": {"last": 109420.0, "min": 109420.0, "max": 109550.0, "mean": 109485.0, "slope": -1.0},
          "rsi14": {"last": 49.5, "min": 47.3, "max": 50.1, "mean": 48.7, "slope": 0.2, "above50_cnt": 1},
          "macd": {"last": -5.0, "signal_last": -6.0, "hist_last": 1.0, "hist_mean": -0.6},
          "atr_pct": {"last": 0.48, "mean": 0.50},
          "h1": {"dist_ema50_pct": -0.20, "hh": 110500.0, "ll": 108100.0, "consec_hist_pos": 2, "hist_slope_3": 0.7}
        },
        "position_state": {
          "status": "flat",
          "size_base": 0.0,
          "size_quote": 0.0,
          "px": 109200.0,
          "lot_size": 0.00001,
          "min_notional": 5.0,
          "exposure_pct": 0.0
        },
        "sizing": {
          "mode": "paliers",
          "fees_bps": 7,
          "usdc_free": 150.0,
          "base_free": 0.0,
          "cap_current_usdc": 200.0,
          "px_est": 109200.0,
          "constraints": {
            "lot_size": 0.00001,
            "min_notional_usdc": 5.0,
            "max_rebalance_pct": 0.30,
            "max_exposure_pct": 0.25,
            "headroom_usdc": 50.0
          },
          "paliers_buy": [
            {"pct": 5,  "budget_plan_usdc": 7.5,  "budget_used_usdc": 7.5,  "qty_base": 0.00006, "notional_usdc": 6.55, "feasible": True,  "why_not": ""},
            {"pct": 10, "budget_plan_usdc": 15.0, "budget_used_usdc": 15.0, "qty_base": 0.00013, "notional_usdc": 14.19, "feasible": True,  "why_not": ""},
            {"pct": 25, "budget_plan_usdc": 37.5, "budget_used_usdc": 37.5, "qty_base": 0.00034, "notional_usdc": 37.13, "feasible": True,  "why_not": ""},
            {"pct": 50, "budget_plan_usdc": 75.0, "budget_used_usdc": 50.0, "qty_base": 0.00046, "notional_usdc": 50.23, "feasible": True,  "why_not": ""},
            {"pct": 75, "budget_plan_usdc": 112.5,"budget_used_usdc": 50.0, "qty_base": 0.00046, "notional_usdc": 50.23, "feasible": True,  "why_not": ""}
          ],
          "paliers_sell": [
            {"pct": 5,  "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 10, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 25, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 50, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"},
            {"pct": 75, "qty_base": 0.0, "notional_usdc": 0.0, "feasible": False, "why_not": "no_position"}
          ],
          "trade_ticket": {"max_buy_usdc": 50.0, "max_buy_qty_base": 0.00046, "max_sell_qty_base": 0.0}
        }
      }
    }
  },
  "memory": {
    "capital": {"initial": 200.0, "current": 200.0, "max_dd_7d": -0.01, "halt_if_below_50pct": True, "halt_triggered": False},
    "capital_history": [{"timestamp": 1756767600, "capital": 200.0}],
    "balances": {
      "USDC": {"amount": 150.0, "value_usdc": 150.0, "px": 1.0},
      "ETH": {"amount": 0.0, "value_usdc": 0.0, "px": 4034.0},
      "BTC": {"amount": 0.0, "value_usdc": 0.0, "px": 109200.0}
    },
    "performance": {"lookback_trades": 10, "winrate": 0.55, "expectancy_r": 0.12, "profit_factor": 1.3, "last_24h_pnl_pct": 0.6},
    "regime": {
      "overview": {"market_state": "range_to_bull", "volatility_rank_30d": 0.55,
        "daily": {"ret_5d_pct": 2.2, "ema30d_slope_pct": 0.8, "atr_rank_30d": 0.55},
        "monthly": {"ret_3m_pct": 12.0, "ret_6m_pct": 28.0, "ret_12m_pct": 65.0, "above_ema12m": True, "ema12m_slope_pct": 8.5}
      },
      "per_asset": {
        "ETHUSDC": {"market_state": "bullish",
          "daily": {"ret_5d_pct": 4.5, "ema30d_slope_pct": 1.6, "atr_rank_30d": 0.58},
          "monthly": {"ret_3m_pct": 18.0, "ret_6m_pct": 45.0, "ret_12m_pct": 80.0, "above_ema12m": True, "ema12m_slope_pct": 10.0},
          "context": {"min_30d": 3600.0, "max_30d": 4200.0, "mean_30d": 3920.0, "avg_volume_30d": 250000.0}
        },
        "BTCUSDC": {"market_state": "neutral",
          "daily": {"ret_5d_pct": 0.4, "ema30d_slope_pct": 0.1, "atr_rank_30d": 0.5},
          "monthly": {"ret_3m_pct": 6.0, "ret_6m_pct": 20.0, "ret_12m_pct": 70.0, "above_ema12m": True, "ema12m_slope_pct": 7.5},
          "context": {"min_30d": 105000.0, "max_30d": 115500.0, "mean_30d": 110300.0, "avg_volume_30d": 4000.0}
        }
      },
      "relative_strength": {"pair": "ETHBTC", "state": "eth_outperform", "h1_slope_pct": 0.9, "d1_slope_pct": 1.4}
    },
    "recent_decisions": [
      {"ts": 1756764000, "symbol": "ETHUSDC", "tf": "1h", "decision": "HOLD", "confidence": 0.6, "entry": "market", "entry_price": 4015.0, "sl": 0.0, "tp": 0.0, "qty_quote": 0.0, "risk_check": "ok", "reason": "Attente confirmation EMA50/RSI>50/MACD>0", "outcome": {"closed": False}, "note": "pre-trade decision"},
      {"ts": 1756760400, "symbol": "BTCUSDC", "tf": "1h", "decision": "HOLD", "confidence": 0.55, "entry": "market", "entry_price": 109050.0, "sl": 0.0, "tp": 0.0, "qty_quote": 0.0, "risk_check": "ok", "reason": "BTC sous EMA50, RSI<50", "outcome": {"closed": False}, "note": "pre-trade decision"}
    ],
    "constraints": {
      "max_exposure_pct": 0.25,
      "max_rebalance_pct": 0.30,
      "stable_symbol": "USDC",
      "symbols": {
        "ETHUSDC": {"lot_size": 0.0001, "min_notional": 5.0},
        "BTCUSDC": {"lot_size": 0.00001, "min_notional": 5.0}
      }
    },
    "sizing": {
      "usdc_free": 150.0,
      "cap_max": 60.0,
      "cap_buy_budget": 50.0,
      "risk_cap": 60.0,
      "risk_pct": 0.30,
      "max_rebalance_pct": 0.30,
      "min_ticket_usdc": 5.0,
      "floor_applied": False
    }
  }
}


In [4]:
from openai import OpenAI

client_openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


decision = get_decision(client_openai, payload)

In [6]:
def _clamp(v, lo, hi):
    try:
        x = float(v)
    except Exception:
        return lo
    return max(lo, min(hi, x))

def _norm_pair(asset: str) -> str:
    return "ETHUSDC" if asset == "ETH" else "BTCUSDC" if asset == "BTC" else "ETHUSDC"

def _truncate(s: str, n: int = 2000) -> str:
    if not isinstance(s, str):
        return ""
    return s if len(s) <= n else (s[:n] + "…")

def _hold_decision(reason: str, sleep_s: int = 43200) -> dict:
    # HOLD canonique (Halt ou fallback erreur)
    return {
        "decision": "HOLD",
        "asset": "USDC",
        "pair": "ETHUSDC",
        "confidence": 1.0,
        "entry": "market",
        "sl": 0.0,
        "tp": 0.0,
        "qty_base": 0.0,
        "risk_check": "ok",
        "reason": reason,
        "next_steps": "Surveiller D1/H4; réactiver si régime s'améliore.",
        "time_sleep_s": int(max(0, sleep_s)),
        "sizing_mode": "paliers",
        "size_pct": None
    }

def _validate_and_normalize(dec: dict) -> dict:
    if not isinstance(dec, dict):
        return _hold_decision("llm_invalid_payload")
    side = str(dec.get("decision", "HOLD")).upper()
    asset = str(dec.get("asset", "USDC")).upper()
    pair  = str(dec.get("pair", _norm_pair(asset))).upper()
    entry = str(dec.get("entry", "market")).lower()
    riskc = str(dec.get("risk_check", "ok")).lower()
    conf  = _clamp(dec.get("confidence", 0.0), 0.0, 1.0)
    sl    = float(dec.get("sl", 0.0) or 0.0)
    tp    = float(dec.get("tp", 0.0) or 0.0)
    tss   = int(dec.get("time_sleep_s", 0) or 0)
    size_pct = dec.get("size_pct", None)
    sizing_mode = dec.get("sizing_mode", "paliers")

    # Cohérence side/asset/pair
    if side not in ("BUY", "SELL", "HOLD"):
        side = "HOLD"
    if side == "HOLD":
        asset = "USDC"
        pair = "ETHUSDC"  # convention unique
        size_pct = 0
        sl = tp = 0.0
    else:
        if asset not in ("ETH", "BTC"):
            # défaut raisonnable : ETH
            asset = "ETH"
        pair = _norm_pair(asset)
        # size_pct requis sur BUY/SELL
        if size_pct not in (0, 5, 10, 25, 50, 75):
            # si manquant, on dégrade en HOLD pour sécurité
            return _hold_decision("size_pct_missing_or_invalid", sleep_s=max(1800, tss))

    # risk_check standardisé
    riskc = "ok" if riskc == "ok" else "too_high"

    # Cadence : bornes (0..21600), on laisse le LLM choisir à l'intérieur
    tss = int(_clamp(tss, 0, 21600))
    # En HOLD sans contexte critique, éviter 0
    if side == "HOLD" and tss < 1800:
        tss = 1800

    reason = _truncate(dec.get("reason", ""))
    next_steps = _truncate(dec.get("next_steps", ""))

    # qty_base ignorée côté exécution paliers, mais on la nettoie
    out = {
        "decision": side,
        "asset": asset,
        "pair": pair,
        "confidence": float(conf),
        "entry": entry if entry in ("market", "limit") else "market",
        "sl": float(sl),
        "tp": float(tp),
        "qty_base": 0.0,  # calculée côté backend si besoin, ignorée en mode paliers
        "risk_check": riskc,
        "reason": reason,
        "next_steps": next_steps,
        "time_sleep_s": tss,
        "sizing_mode": sizing_mode if sizing_mode in ("paliers", "legacy") else "paliers",
        "size_pct": int(size_pct) if size_pct in (5, 10, 25, 50, 75) else None
    }
    return out

# -------- 2) Court-circuit HALT --------
mem = payload.get("memory", {}) if isinstance(payload, dict) else {}
cap = mem.get("capital", {}) if isinstance(mem, dict) else {}


# -------- 3) Appel LLM protégé --------
try:
    raw_decision = get_decision(client_openai, payload)
    decision = _validate_and_normalize(raw_decision)
except Exception as e:
    decision = _hold_decision(f"llm_error:{type(e).__name__}")


print("===================================")
print(f"Décision : {decision}")

Décision : {'decision': 'BUY', 'asset': 'ETH', 'pair': 'ETHUSDC', 'confidence': 0.8, 'entry': 'market', 'sl': 0.0, 'tp': 0.0, 'qty_base': 0.0, 'risk_check': 'ok', 'reason': 'Convergence H1/H4 sur ETH : 1) Momentum solide avec MACD histo positif et en hausse (hist_last 5.4, hist_slope_3=0.6, consécutifs +8), 2) Tendance forte : EMA20/50 toutes deux en progression (slope 1.25/1.05), prix en permanence au-dessus des moyennes mobiles, RSI14 élevé et stable > 60, force récente avec +2.4 % sur 3h et +4.1 % sur 12h, 9 bougies vertes sur 12. Volatilité (ATR_pct 0.87 ≈ moyenne) reste modérée et capital stable (DD7j -0.01 %, equity=haut, vol maîtrisée). Regime daily/monthly très favorable à ETH (ret_5j=4.5 %, slope_ema30d=1.6, monthly=+18 à +80 %). Allocation BTC non justifiée (BTC sous EMA, RSI < 50, pas de momentum, neutral state). Paliers 25%, 10% et 5% faisables ; sélection du plus large faisable (25 %, notional 37.1 USDC, fully compliant) mais borné par headroom/max_rebalance, donc palier 2

In [8]:
print("===================================")
print("Décision reçue (brève) →",
    f"side={decision.get('decision')} asset={decision.get('asset')} size_pct={decision.get('size_pct')}")

# ------- court-circuits sûrs -------
cap = memory.get("capital", {})

side = str(decision.get("decision", "HOLD")).upper()

try:
    binance_time_offset_ms(client)  # best effort
except Exception:
    pass

# ------- exécution (paliers, contrôles internes dans execute_trade) -------
try:
    memory = execute_trade(client, decision, memory)
except Exception as e:
    # On journalise un SKIPPED d'urgence si execute_trade a levé (réseau, symbole, etc.)
    try:
        memory = update_memory_with_decision(
            memory=memory,
            symbol=decision.get("pair", "ETHUSDC"),
            tf=decision.get("tf", "1h"),
            decision_dict={**decision, "decision": "HOLD", "reason": f"action_error:{type(e).__name__}"},
            price_usdc=0.0,
            qty_quote=0.0,
            note="action_task fallback",
            keep_last=20
        )
    except Exception:
        pass
    print(f"[action_task] Erreur exécution : {type(e).__name__} → skip.")


# ------- cadence : on ne bloque pas un worker Airflow avec sleep -------
sleep_s = int(decision.get("time_sleep_s", 0) or 0)
if sleep_s > 0:
    print(f"[Décideur] time_sleep_s suggéré : {sleep_s}s (aucun sleep ici ; la cadence est gérée par Airflow).")

Décision reçue (brève) → side=BUY asset=ETH size_pct=25
[TimeSync] server-local offset = 2290 ms
